## Dydaktyczny material "Aplikacyjny interfejs programistyczny zgodny ze specyfikacją REST (RESTful API)"

<h3> I część. Przygotowanie </h3> 

Aby móc korzystać z bazy danych MySQL w ramach mikroserwisu, niezbędne jest zainstalowanie odpowiedniego oprogramowania serwera bazodanowego MySQL. Do wyboru są dwie popularne opcje:

- XAMPP - pakiet obsługujący MySQL, Apache i inne komponenty w jednej instalacji.
- MySQL Workbench - dedykowane narzędzie firmy Oracle do administracji serwerem MySQL.

Następnie, jeśli został wybrany XAMPP, przed rozpoczęciem pracy nad mikroserwisem należy uruchomić moduły Apache oraz MySQL w panelu kontrolnym XAMPP. Umożliwi to działanie lokalnego serwera bazodanowego MySQL oraz serwera WWW Apache.

Kolejny krok to zainicjalizowanie struktury bazy danych poprzez wykonanie skryptu SQL z pliku init.sql (lub innej nazwy). Plik ten zawiera instrukcje SQL tworzące tabelę(-e) wraz z kluczami, indeksami itp. Wykonanie tego skryptu stworzy odpowiednią strukturę bazy danych dla mikroserwisu.

```
--
-- Struktura tabeli `room-db`
--

CREATE TABLE `room-db` (
  `id` int(100) NOT NULL,
  `token` varchar(255) NOT NULL,
  `password` varchar(255) DEFAULT NULL,
  `creation_time` timestamp NOT NULL DEFAULT current_timestamp()
) ENGINE=InnoDB DEFAULT CHARSET=latin1 COLLATE=latin1_swedish_ci;

--
-- Zrzut danych tabeli room-db
--

INSERT INTO `room-db` (`id`, `token`, `password`, `creation_time`) VALUES
(1, 'ydhxr', 'hh', '2023-10-22 12:46:00'),
(2, '4Lif4', '   ', '2023-10-22 12:46:24'),
(3, 'h7I79', '', '2023-10-22 12:47:14'),
(4, 're9mu', '', '2023-10-22 13:03:49'),
(5, 'vam4m', '', '2023-10-22 13:04:16'),
(6, 'RZIpe', '', '2023-10-22 13:04:20'),
(7, 'YhgGo', '', '2023-10-22 13:44:12'),
(8, 'SsNOb', 'SuperRoom', '2023-10-25 11:08:58'),
(9, '5kduf', '', '2023-10-25 13:29:14'),
(10, 'Md8hV', '', '2023-11-11 12:07:14'),
(11, 'UutRg', '1234567', '2023-11-11 12:18:35'),
(12, 'G1Rmh', '', '2023-12-16 13:39:33'),
(13, 'OKWuu', 'pretty', '2023-12-16 14:55:05'),
(14, '0ePAd', '', '2023-12-18 20:14:45'),
(15, 'N5u34', 'the-sun', '2023-12-19 17:41:19'),
(16, '08fuk', 'the-sun', '2023-12-19 17:41:53'),
(17, 'aEmPk', 'kk', '2023-12-19 17:42:01'),
(18, 'WTZcI', 'kk', '2023-12-19 17:42:24'),
(19, 'ow7Xg', 'moon', '2023-12-19 17:42:43'),
(20, 'tiken', 'pass', '2023-12-19 17:56:50'),
(21, 'hard', 'pass-3', '2023-12-19 19:19:39'),
(22, 'rock', '', '2023-12-19 19:20:29'),
(23, 'EBhkN', '', '2023-12-19 19:21:52'),
(24, 'Ixhx6', 'solveIt', '2023-12-19 19:22:10'),
(25, 'kolo', 'promien', '2023-12-19 19:27:59'),
(26, 'xqmS6', '', '2023-12-23 14:53:09');

ALTER TABLE `room-db`
  ADD PRIMARY KEY (`id`);

ALTER TABLE `room-db`
  MODIFY `id` int(100) NOT NULL AUTO_INCREMENT, AUTO_INCREMENT=27;
COMMIT;
```

Dalej należy zainstalować wszystkie biblioteki Pythona potrzebne dla tego API REST opartego o Flaska za pomocą następujących poleceń pip:

* pip install flask - instaluje główną bibliotekę Flask
* pip install flask_restx - instaluje Flask-RESTX do budowy API REST
* pip install mysql-connector-python - instaluje bibliotekę łączącą Pythona z MySQL
* pip install datetime - instaluje moduł datetime z biblioteki standardowej Pythona
* pip install random - instaluje moduł random z biblioteki standardowej Pythona
* pip install string - instaluje moduł string z biblioteki standardowej Pythona

<h3> II część. Inicjalizacja </h3> 
Ten fragment kodu definiuje podstawową konfigurację aplikacji Flask, łączącej się z bazą danych MySQL. 
Zostały importowane różne moduły:

- datetime: umożliwia pracę z datami i czasem;

In [113]:
from datetime import datetime

- Resource z flask_restx: jest to klasa, którą można rozszerzyć, aby utworzyć zasób RESTful w aplikacji Flask;

In [114]:
from flask_restx import Resource

- Flask: jest to klasa reprezentująca główną aplikację Flask, gdzie :
    - render_template: umożliwia renderowanie szablonów HTML,
    - request: udostępnia dostęp do danych przesyłanych w żądaniach HTTP,
    - jsonify: pomocny w zwracaniu odpowiedzi w formacie JSON;

In [115]:
from flask import Flask, render_template, request, jsonify

- random i string: służą do generowania losowych ciągów znaków;

In [116]:
import random
import string

- mysql.connector: moduł do obsługi połączenia z bazą danych MySQL;

In [117]:
import mysql.connector

- flask_restx: jest rozszerzeniem dla frameworka Flask, który ułatwia tworzenie interfejsów API w stylu RESTful w aplikacjach opartych na Flask. Z niego są importowane:
    -   Api: klasa dostarczana przez Flask-RESTx, która ułatwia definiowanie i konfigurowanie interfejsu API dla aplikacji opartej na Flask;
    -  fields: moduł w Flask-RESTx, który umożliwia definiowanie pól danych używanych w modelach API;
    - reqparse: moduł umożliwiający analizę i walidację argumentów przekazywanych w żądaniach HTTP, takich jak parametry zapytań, nagłówki czy ciała żądania. 

In [118]:
from flask_restx import Api, fields, reqparse

Dalej została stworzona instancja klasy Flask, reprezentująca aplikację. W Pythonie, zmienna __name__ jest wbudowaną zmienną specjalną, która jest używana do określenia kontekstu, w jakim jest uruchamiany skrypt.

In [119]:
app = Flask(__name__)

Następnie definiuje się konfiguracja połączenia z bazą danych MySQL. Parametry obejmują adres hosta, nazwę użytkownika, hasło i nazwę bazy danych.

In [120]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'test'
}

Dalej zostały zdefiniowane i skonfigurowane funkcje związane z Flask-RESTx, które są używane do budowania RESTful mikroserwisów w aplikacji Flask. Na początku został inicjalizowany obiekt Api z Flask-RESTx, który reprezentuje całe API. Określa on wersję, tytuł i opis API. 

In [121]:
api = Api(
    version='1.0',
    title='Room API',
    description='API Documentation',
)

Zatem jest inicjalizuja obiektu api w kontekście aplikacji Flask (app). To pozwala na integrację Flask-RESTx z aplikacją.

In [122]:
api.init_app(app)

Zatem jest tworzenie przestrzeni nazw dla operacji na tokenach. Przestrzeń nazw (namespace) to koncept w Flask-RESTx, który pomaga w organizacji endpointów i zasobów. W tym przypadku, przestrzeń nazw jest związana z operacjami HTTP dotyczącymi tokenów.

In [123]:
api = api.namespace('', description='HTTP Operations related to tokens')

Model danych token_model definiuje strukturę danych, która będzie używana do reprezentacji tokenów w API. To jest używane do serializacji i deserializacji danych związanych z tokenem.

In [124]:
token_model = api.model('Token', {
    'token': fields.String(required=True, description='Token value'),
    'password': fields.String(required=False, description='Password associated with the token'),
})

 Model token_model_full pomimo informacji ogólnej (zawartej w token_model), uwzględnia dodatkowe informacje związane z hiperłączami. Model zawiera zagnieżdżony model Links, który składa się z pól self, update i delete typu string, reprezentujących odpowiednie hiperłącza. Taka struktura modelu pozwala na precyzyjne zdefiniowanie formatu danych, które API oczekuje i zwraca, co ułatwia zarządzanie danymi w ramach aplikacji. Dodatkowo, to podejście wspiera zgodność z zasadą HATEOAS (Hypertext As The Engine Of Application State), umożliwiając reprezentację hiperłączy w odpowiedziach API.

In [125]:
links_model = api.model('Links', {
    'self': fields.String,
    'update': fields.String,
    'delete': fields.String
})

token_model_full = api.model('Token_full', {
    'id': fields.Integer,
    'token': fields.String,
    'password': fields.String,
    'creation_time': fields.String,
    '_links': fields.Nested(links_model)
})

Parser token_parser jest używany do parsowania danych wejściowych z żądań HTTP. Definiuje oczekiwane argumenty, takie jak token i password, które mogą być przekazywane w żądaniu.

In [126]:
token_parser = reqparse.RequestParser()
token_parser.add_argument('token', type=str, help='Token value')
token_parser.add_argument('password', type=str, help='Password associated with the token')

W kontekście RESTful mikroserwisów, te elementy pomagają w definiowaniu struktury API, obsłudze danych wejściowych, oraz w dokumentowaniu operacji dostępnych w API. Modele danych i parsery ułatwiają również walidację i przetwarzanie danych przesyłanych przez klientów. Przestrzenie nazw pozwalają na logiczne grupowanie operacji, co poprawia czytelność i organizację kodu.

<h3>III część. Definicja punktów końcowych </h3>

Dalej zostanie dodany punkt końcowy 'generate-token', który będzie obsługiwał:

1. Żądanie GET:

- Kiedy użytkownik otwiera przeglądarkę i wpisuje ścieżkę /generate-token, wykonuje się obsługa żądania GET.
- W tym przypadku, funkcja generate_and_insert_token inicjalizuje zmienne password i token jako puste ciągi znaków.
- Następnie renderuje szablon HTML (index.html) za pomocą funkcji render_template. Szablon ten może korzystać z wartości token i password.
- Ostatecznie, funkcja zwraca zawartość HTML jako odpowiedź na żądanie GET.

2. Żądanie POST:

- Kiedy użytkownik wysyła formularz na stronie /generate-token (na przykład, po wypełnieniu pola hasła i wysłaniu formularza), wykonuje się obsługa żądania POST.
- Funkcja pobiera wartość hasła z formularza za pomocą request.form.get('password').
- Następnie generuje nowy token za pomocą funkcji generate_token i wykonuje wstawienie danych (token i hasło) do bazy danych za pomocą funkcji insert_data_into_db.
-Ostatecznie, funkcja renderuje szablon HTML i zwraca go jako odpowiedź na żądanie POST.

In [127]:
@app.route('/generate-token', methods=['GET', 'POST'])
def generate_and_insert_token():
    password = ""  # Initialize the password variable
    token = ""

    if request.method == 'POST':
        password = request.form.get('password')  # Retrieve the password from the form
        token = generate_token()
        insert_data_into_db(token, password)

    html_content = render_template('index.html', token=token, password=password)
    return html_content

Funkcje Pomocnicze:

- generate_token: funkcja generująca losowy token składający się z liter i cyfr;

In [128]:
def generate_token():
    token = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(5))
    return token

- insert_data_into_db: odpowiada za wstawienie danych (tokenu i hasła) do bazy danych MySQL. Przed dokonaniem wstawienia, funkcja sprawdza, czy token o podanym identyfikatorze już istnieje, a następnie wykonuje odpowiednie zapytanie SQL w zależności od tego, czy podano identyfikator. W przypadku sukcesu, dane są zatwierdzane, a połączenie z bazą danych zamykane. W przypadku błędu, funkcja zgłasza wyjątek zawierający informacje o wystąpieniu błędu MySQL;

In [129]:
def insert_data_into_db(token, password, token_id=None):
    try:
        if token:
            conn, cursor = connect_to_database()

            if token_id is not None and token_id != 0:

                cursor.execute("SELECT COUNT(*) FROM `room-db` WHERE id = %s", (token_id,))
                result = cursor.fetchone()

                if result and result['COUNT(*)'] > 0:
                    raise ValueError(f"Token with ID {token_id} already exists")

            if token_id is not None and token_id != 0:
                insert_query = """
                    INSERT INTO `room-db` (`id`, `token`, `password`)
                    VALUES (%s, %s, %s)
                """
                cursor.execute(insert_query, (token_id, token, password))
            else:
                insert_query = """  
                    INSERT INTO `room-db` (`token`, `password`)
                    VALUES (%s, %s)
                """
                cursor.execute(insert_query, (token, password))

            conn.commit()
            close_database_connection(cursor, conn)

        else:
            raise ValueError("Error: 'token' is required")

    except mysql.connector.Error as e:
        print("Error:", e)

- connect_to_database i close_database_connection: funkcje do nawiązywania i zamykania połączenia z bazą danych.

In [130]:
def connect_to_database():
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor(dictionary=True)
    return conn, cursor


def close_database_connection(cursor, conn):
    cursor.close()
    conn.close()

Dalej uruchamiamy serwer Flask, który będzie nasłuchiwał na żądania HTTP na domyślnym adresie http://127.0.0.1:5000/. 
Poniższy kod sprawdza, czy skrypt Pythona jest uruchamiany jako plik główny (a nie zaimportowany jako moduł do innego skryptu). Jeśli ten warunek jest spełniony, co oznacza, że skrypt jest uruchamiany bezpośrednio, a nie importowany, to następuje uruchomienie aplikacji Flask.


In [140]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Jan/2024 22:04:32] "DELETE /token/16 HTTP/1.1" 404 -
127.0.0.1 - - [15/Jan/2024 22:04:39] "DELETE /token/17 HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 22:04:50] "GET /tokens HTTP/1.1" 200 -


Teraz zdefiniujemy nowy punkt końcowy `/token/<int:id>`, który będzie obsługiwał trzy metody HTTP: GET, POST, DELETE i PUT. Oto opis poszczególnych części kodu:

1. Żądanie GET:

- Jeśli otrzymano żądanie GET na ścieżce `/token/<int:id>`, funkcja display_or_delete_or_update_token_info(id) wywołuje funkcję retrieve_token_info(id) w celu pobrania informacji o tokenie o określonym ID z bazy danych.
- Jeśli token zostanie znaleziony, renderuje się szablon HTML (token_info.html), a następnie zwraca odpowiedź z informacjami o tokenie.
- Jeśli token nie zostanie znaleziony, zwracany jest komunikat o błędzie 404.
- W kodzie jest dodatkowe sprawdzenie, czy żądanie HTTP nie jest typu DELETE. Ma to związek z tym, że identyfikator konkretnego zasobu (np. tokenu) jest przekazywany w adresie URL jako parametr. Normalnie żądanie GET do takiego adresu powinno zwrócić szczegóły danego tokenu. Jednak dodatkowo do tego adresu URL jest podpięty przycisk "Usuń" służący do wywołania żądania DELETE w celu usunięcia tokenu. Problem polega na tym, że sam przycisk z adresem /token/<id> wywołuje żądanie GET zamiast DELETE. Dlatego w kodzie musi być sprawdzenie, czy przypadkiem żądanie nie ma ustawionej metody DELETE (jest to możliwe w Flasku). Jeśli tak, to zamiast zwrócić dane tokenu, zostanie on usunięty z bazy danych.

2. Żądanie POST:

- Jeśli otrzymano żądanie POST na ścieżce `/token/<int:id>`, funkcja próbuje zaktualizować hasło i token dla określonego ID na podstawie danych przesłanych w formularzu.
- Jeśli operacja aktualizacji powiedzie się, zwracany jest komunikat potwierdzający aktualizację.
- Jeśli token o podanym ID nie zostanie znaleziony, zwracany jest komunikat o błędzie 404.

3. Żądanie DELETE:

- Jeśli otrzymano żądanie DELETE na ścieżce `/token/<int:id>`, funkcja próbuje usunąć token o określonym ID z bazy danych.
- Jeśli operacja usunięcia powiedzie się, zwracany jest komunikat potwierdzający usunięcie.
- Jeśli token o podanym ID nie zostanie znaleziony, zwracany jest komunikat o błędzie 404.

4. Żądanie PUT:
- Gdy serwer otrzymuje żądanie PUT na ścieżce `/token/<int:id>`, sprawdza, czy metoda to PUT. To żądanie umożliwia modyfikację danych związanych z tokenem w bazie danych.
- Następnie pobiera nowy token i nowe hasło z formularza żądania. Jeśli aktualizacja informacji dla określonego identyfikatora tokena przy użyciu funkcji update_token_info powiedzie się, serwer zwraca komunikat informujący, że informacje dla danego identyfikatora zostały zaktualizowane. W przeciwnym razie zwracany jest komunikat o nieznalezieniu wiersza o podanym identyfikatorze, wskazujący na błąd 404.

W kodzie implementującym endpoint `/token/<int:id>`, zastosowano dodatkową warstwę sprawdzającą argumenty metody HTTP. Jest to konieczne ze względu na domyślne zachowanie w bibliotece Flask, gdzie w odpowiedzi na interakcję z przyciskami formularza zawsze wysyłane są żądania typu GET lub POST. Jednakże, w kontekście aktualizacji, usuwania lub dodawania nowych zasobów, które wymagają użycia metod PUT, DELETE lub POST, konieczne jest wprowadzenie mechanizmu pozwalającego na przesłanie tych konkretnych metod.

W związku z tym, do każdego formularza (w pliku tokens.html) została dodana ukryta zmienna `_method`, która zawiera informację o zamierzonej metodzie HTTP. Mechanizm ten umożliwia symulację wysłania żądania z wybraną metodą, taką jak DELETE. To podejście pozwala na skorzystanie z dostępnych funkcji w Flask, jednocześnie umożliwiając precyzyjne określenie żądanej metody HTTP, co jest szczególnie istotne w przypadku operacji, które wykraczają poza standardowe GET i POST.

In [131]:
@app.route('/token/<int:id>', methods=['GET', 'PUT', 'POST', 'DELETE'])
def display_or_delete_or_update_token_info(id):
    method = request.args.get('_method', None)
    if request.method == 'GET' and method != 'DELETE':
        # Retrieve and display token information
        token, password, creation_time = retrieve_token_info(id)
        if token is not None:
            return render_template('token_info.html', id=id, token=token, password=password,
                                   creation_time=creation_time)
        else:
            return "Row with ID {} not found.".format(id), 404

    elif request.method == 'PUT' :
        # Update the password and the token for the specified token ID
        new_token = request.form.get('token')
        new_password = request.form.get('password')
        if update_token_info(id, new_token, new_password):
            return "Information for ID {} has been updated.".format(id)
        else:
            return "Row with ID {} not found.".format(id), 404

    elif request.method == 'POST':
        new_token = request.form.get('token')
        new_password = request.form.get('password')
        if new_token:
            insert_data_into_db(new_token, new_password, id)
            return "New token added successfully."
        else:
            return "Missing 'token' parameter in the request.", 400

    elif request.method == 'DELETE' or method == 'DELETE':
        # Delete the row with the specified id
        if delete_token(id):
            return "Row with ID {} has been deleted.".format(id)
        else:
            return "Row with ID {} not found.".format(id), 404

Funkcje Pomocnicze:

- retrieve_token_info(id): funkcja pobierająca informacje o tokenie na podstawie ID z bazy danych;

In [132]:
def retrieve_token_info(id):
    try:
        conn, cursor = connect_to_database()

        # Query the database to retrieve the token, password, and creation_time for the specified id
        cursor.execute("SELECT token, password, creation_time FROM `room-db` WHERE id = %s", (id,))
        row = cursor.fetchone()

        if row:
            token = row['token']
            password = row['password']
            creation_time = row['creation_time']
            return token, password, creation_time

    except mysql.connector.Error as e:
        print("Error:", e)
    finally:
        close_database_connection(cursor, conn)

    return None, None, None  # Return None if the row is not found or an error occurs

- delete_token(id): funkcja usuwająca token na podstawie ID z bazy danych;

In [133]:
def delete_token(id):
    try:
        conn, cursor = connect_to_database()

        # Execute a DELETE query to remove the row with the specified id
        cursor.execute("DELETE FROM `room-db` WHERE id = %s", (id,))

        # Check if any rows were affected
        if cursor.rowcount > 0:
            conn.commit()
            close_database_connection(cursor, conn)
            return True
        else:
            # No rows were affected, indicating the ID was not found
            close_database_connection(cursor, conn)
            return False

    except mysql.connector.Error as e:
        print("Error:", e)
        return False

- update_token_info(id, new_token, new_password, new_time=None): Funkcja aktualizująca token na podstawie ID z nowymi danymi.

In [134]:
def update_token_info(id, new_token, new_password, new_time=None):
    try:
        conn, cursor = connect_to_database()

        # Determine the new creation time
        if new_time is None:
            new_time = datetime.now()

        # Execute an UPDATE query to change the token, password, and time for the specified ID
        cursor.execute("UPDATE `room-db` SET token = %s, password = %s, creation_time = %s WHERE id = %s",
                       (new_token, new_password, new_time, id))

        # Check if any rows were affected
        if cursor.rowcount > 0:
            conn.commit()
            close_database_connection(cursor, conn)
            return True
        else:
            # No rows were affected, indicating the ID was not found
            close_database_connection(cursor, conn)
            return False

    except mysql.connector.Error as e:
        print("Error:", e)
        return False

Żeby zaimplementować taką samą funkcjonalność, jaką spełnia powyższy kod, ale który by miał dokumentacje API za pomocą Swaggera musimy:

1. W podobny sposób zadeklarować ściężkę API, ale odwolując do obiektu biblioteki Flask_Restx (czyli do api). URL będzie z prefiksem /api.
2. Zdefiniować clasę TokenResource - klasa ta dziedziczy po klasie Resource z Flask-RESTx, co umożliwia obsługę różnych metod HTTP (GET, POST, DELETE, PUT) dla zasobu tokena.
3. Zdefiniować metody get(self, id), post(self, id), delete(self, id) i put(self, if), które obslugują swoje żądania.


Anotacja `@api.expect(token_parser)` w Flask-RESTx oznacza, że dany endpoint API oczekuje, że dane przesłane w żądaniu będą zgodne z formatem zdefiniowanym przez parser.

Kod ten używa Flask-RESTx do zdefiniowania struktury API, obsługi żądań HTTP i walidacji danych wejściowych za pomocą parsera. Jest to bardziej rozbudowane podejście do budowania API w porównaniu do standardowego Flask, pozwalające na bardziej zorganizowaną obsługę zasobów i działań związanych z nimi.


In [135]:
@api.route('/api/token/<int:id>')
class TokenResource(Resource):
    def get(self, id):
        token, password, creation_time = retrieve_token_info(id)
        if token is not None:
            # Convert date to string before returning response
            creation_time_str = creation_time.strftime("%Y-%m-%d %H:%M:%S") if creation_time else None
            return {'id': id, 'token': token, 'password': password, 'creation_time': creation_time_str}
        else:
            api.abort(404, f"Row with ID {id} not found")

    @api.expect(token_parser)
    def post(self, id):
        args = token_parser.parse_args()
        new_token = args['token']
        new_password = args['password']
        if new_token:
            insert_data_into_db(new_token, new_password, id)
            return f"The row for ID {id} has been added."
        else:
            api.abort(400, "Missing 'token' parameter in the request.")

    @api.expect(token_parser)
    def put(self, id):
        args = token_parser.parse_args()
        new_token = args['token']
        new_password = args['password']
        if update_token_info(id, new_token, new_password):
            return f"Information for ID {id} has been updated."
        else:
            api.abort(404, f"Row with ID {id} not found")

    def delete(self, id):
        if delete_token(id):
            return f"Row with ID {id} has been deleted."
        else:
            api.abort(404, f"Row with ID {id} not found")

Ten fragment kodu definiuje funkcję obsługującą żądanie HTTP GET dla ścieżki   `/tokens`. Dana ścieżka oferuje parametry `filter_by` i `search_value`. Parametr `filter_by` oznacza kolumnę, według której zostanie przeprowadzona operacja filtrowania, a `search_value` jest wartością, według której filtrowane są wyniki w określonej kolumnie (używane z parametrem `filter_by`). Jeśli oba parametry są dostępne, dodawane jest warunkowe WHERE do zapytania, filtrujące wyniki według podanego kryterium.

Przed zwróceniem wyników, tokeny są formatowane do postaci zgodnej z HATEOAS - każdy obiekt tokenu zawiera dodatkowe pole `_links` ze zdefiniowanymi adresami URL (linkami) umożliwiającymi wykonanie operacji na danym tokenie (self, update, delete).
Linki te pozwalają klientowi odnaleźć i wykonać konkretne akcje poprzez samodokumentujący się interfejs, bez znajomości szczegółów implementacji backendu. Jest to realizacja idei HATEOAS w REST API. Dzięki temu frontend aplikacji może być zaimplementowany w sposób niezależny od backendu i łatwo dostosowany do zmian - wystarczy że będzie odczytywał linki zwrócone razem z danymi.

In [136]:
@app.route('/tokens', methods=['GET'])
def display_filtered_tokens():
    try:
        conn, cursor = connect_to_database()

        # Default query to retrieve all data
        query = "SELECT id, token, password, creation_time FROM `room-db`"

        # Check if filter_by and search_value parameters are provided
        filter_by = request.args.get('filter_by')
        search_value = request.args.get('search_value')

        # If both filter_by and search_value are provided, add a WHERE clause to the query
        if filter_by and search_value:
            query += f" WHERE `{filter_by}` LIKE '%{search_value}%'"

        # Execute the query
        cursor.execute(query)
        data = cursor.fetchall()

        # Format each token data with links
        formatted_data = []
        for token in data:
            formatted_token = {
                'id': token['id'],
                'token': token['token'],
                'password': token['password'],
                'creation_time': token['creation_time'].isoformat(),
                '_links': {
                    'self': f"http://localhost:5000/token/{token['id']}",
                    'update': f"http://localhost:5000/token/{token['id']}",
                    'delete': f"http://localhost:5000/token/{token['id']}",
                }
            }
            formatted_data.append(formatted_token)

        return render_template('tokens.html', data=formatted_data)

    except mysql.connector.Error as e:
        print("Error:", e)
    finally:
        close_database_connection(cursor, conn)

    return "An error occurred while fetching token data."

Podobny kod, ale dla Swaggera ma poniższą postać. Taki kod jest bardziej skoncentrowany na dostarczaniu API RESTful i zwracaniu danych w formacie JSON, co jest typowe dla zastosowań, w których klientem jest np. aplikacja frontendowa w JavaScript.

In [137]:
@api.route('/api/tokens')
class TokenList(Resource):
    @api.doc(responses={200: 'OK'})
    @api.marshal_with(token_model_full, as_list=True)
    def get(self):
        try:
            conn, cursor = connect_to_database()

            # Default query to retrieve all data
            query = "SELECT id, token, password, creation_time FROM `room-db`"

            # Check if filter_by and search_value parameters are provided
            filter_by = request.args.get('filter_by')
            search_value = request.args.get('search_value')

            # If both filter_by and search_value are provided, add a WHERE clause to the query
            if filter_by and search_value:
                query += f" WHERE `{filter_by}` LIKE '%{search_value}%'"

            # Execute the query
            cursor.execute(query)

            # Fetch the results after executing the query
            tokens = cursor.fetchall()

            # Convert the creation_time to ISO format for each token
            results = []
            for token in tokens:
                token_data = {
                    'id': token['id'],
                    'token': token['token'],
                    'password': token['password'],
                    'creation_time': token['creation_time'].isoformat(),
                    '_links': {
                        'self': f"http://localhost:5000/token/{token['id']}",
                        'update': f"http://localhost:5000/token/{token['id']}",
                        'delete': f"http://localhost:5000/token/{token['id']}",
                    }
                }
                results.append(token_data)

            close_database_connection(cursor, conn)
            return results, 200

        except mysql.connector.Error as e:
            print("Error:", e)
            return {"error": "An error occurred while fetching token data."}, 500

Komunikacja mikroseriwsu Room z mikroserwisem Student jest za pomocą punktu końcowego /get-data/<string:token>. Funkcja tego puktu, get_token_data(token) wykorzystuje funkcję retrieve_token_info_by_token(token) w celu pobrania informacji o tokenie z bazy danych. Jeśli dane są dostępne, formatuje je jako słownik i zwraca w formie JSON za pomocą jsonify(response_data). W przypadku braku danych zwraca komunikat o błędzie w formie JSON i kod odpowiedzi 404. Jest to rozwiązanie mniej zaawansowane, aniż następny punkt końcowy, który oferuję rozwiązanie Swagger, chociaż kod jest prawie analogiczny.

In [138]:
@app.route('/get-data/<string:token>', methods=['GET'])
def get_token_data(token):
    data = retrieve_token_info_by_token(token)
    if data:
        response_data = {
            'token': data[0],  # Assuming data[0] contains the token
            'password': data[1]  # Assuming data[1] contains the password
        }
        return jsonify(response_data)
    else:
        return jsonify({'error': 'Token not found'}), 404


@api.route('/api/get-data/<string:token>')
class TokenResource(Resource):
    @api.marshal_with(token_model)
    def get(self, token):
        data = retrieve_token_info_by_token(token)
        if data:
            response_data = {
                'token': data[0],  # Assuming data[0] contains the token
                'password': data[1]  # Assuming data[1] contains the password
            }
            return response_data
        else:
            api.abort(404, 'Token not found')


def retrieve_token_info_by_token(token):
    try:
        conn, cursor = connect_to_database()

        # Query the database to retrieve the token, password, and creation_time for the specified token
        cursor.execute("SELECT token, password FROM `room-db` WHERE token = %s", (token,))
        row = cursor.fetchone()

        if row:
            token = row['token']
            password = row['password']
            return token, password

    except mysql.connector.Error as e:
        print("Error:", e)
    finally:
        close_database_connection(cursor, conn)

    return None, None  # Return None if the row is not found or an error occurs